In [3]:
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=375d30e42d33a5c26146c761f9e1f52410b242feec2dd3e91f3b6a0786a0ce5f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [13]:
import datasets

In [14]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

train_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'train[:200000]')
validation_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'validation')
test_dataset = load_dataset('ai4bharat/naamapadam', lang , split = 'test')



In [15]:
features = train_dataset.features
print(features)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [16]:
text_column_name = "tokens"
label_column_name = "ner_tags"

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [17]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np
config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER", num_labels=num_labels)

In [18]:
# Tokenize all texts and align the labels with them.
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding="max_length",
        truncation=True,
        max_length=128,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    # print(tokenized_inputs)
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    # print(tokenized_inputs)
    return tokenized_inputs

In [19]:
correct_output = tokenize_and_align_labels(train_dataset[0:1])
tokens = tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0])
print(tokens)

# Now we have input ids and labels to pass to model

['[CLS]', 'सक', '##टर', '55', '/', '56', 'क', 'एस', '##ए', '##च', '##ओ', 'अ', '##र', '##वि', '##द', 'कमार', 'न', 'ब', '##ता', '##या', 'कि', 'इस', 'मा', '##मल', 'म', 'आई', '##पी', '##सी', 'की', 'ध', '##ारा', '376', '-', 'डी', '(', 'ग', '##गर', '##प', ')', 'क', 'त', '##हत', 'मा', '##मल', '##ा', 'दर', '##ज', 'कर', 'लिया', 'गया', 'ह', '।', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [20]:
for token,label in zip(tokenizer.convert_ids_to_tokens(correct_output["input_ids"][0]) , correct_output["labels"][0]):
  print(f"{token:_<40}{label}")


[CLS]___________________________________-100
सक______________________________________0
##टर____________________________________-100
55______________________________________0
/_______________________________________-100
56______________________________________-100
क_______________________________________0
एस______________________________________0
##ए_____________________________________-100
##च_____________________________________-100
##ओ_____________________________________-100
अ_______________________________________1
##र_____________________________________-100
##वि____________________________________-100
##द_____________________________________-100
कमार____________________________________2
न_______________________________________0
ब_______________________________________0
##ता____________________________________-100
##या____________________________________-100
कि______________________________________0
इस______________________________________0
मा______________________________________

In [21]:
tokenized_train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_train_dataset)

  0%|          | 0/200 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 200000
})


In [22]:
# Problem is fixed now!!
print(len(tokenized_train_dataset[0]['ner_tags']))
print(len(tokenized_train_dataset[0]['labels']))
print(len(tokenized_train_dataset[0]['input_ids']))

30
128
128


In [23]:
# Test Dataset
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_test_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})


In [24]:
# Validation DataSet
tokenized_validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
)
print(tokenized_validation_dataset)

  0%|          | 0/14 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13460
})


In [25]:
# Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
# Metric to see how my model is performing For NER we use seqeval
metric = datasets.load_metric("seqeval")

In [27]:
# Seeing the working of seqeval
example_text = train_dataset[0]
label_list = train_dataset.features["ner_tags"].feature.names
label_list

labels = [label_list[i] for i in example_text["ner_tags"]]
print(labels)

metric.compute(predictions=[labels] , references=[labels])

['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


{'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [28]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
    }

In [33]:
# Define Training Args
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
)

In [34]:
trainer = Trainer(
   model,
   args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_validation_dataset,
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [35]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.213700,0.213512,0.706477,0.733445,0.719708,0.933005
2,0.177300,0.183488,0.754552,0.769810,0.762105,0.942342


TrainOutput(global_step=40000, training_loss=0.2221855836868286, metrics={'train_runtime': 6943.5531, 'train_samples_per_second': 57.607, 'train_steps_per_second': 5.761, 'total_flos': 2.61308568576e+16, 'train_loss': 0.2221855836868286, 'epoch': 2.0})

In [36]:
# Save Model
trainer.save_model("model_indic_ner_2Epoch_two_lakh")

In [37]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    ___
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    ___
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [38]:
zip_dir()

/kaggle/working/directory.zip

In [44]:
# Evaluation of Model
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)
specific_results = {}
for key in metrics:
    if 'test_precision' in key:
      specific_results['test_precision'] = metrics[key]
    elif 'test_recall' in key:
      specific_results['test_recall'] = metrics[key]
    elif 'test_f1' in key:
      specific_results['test_f1'] = metrics[key]
print(specific_results)

{'test_precision': 0.7797270955165692, 'test_recall': 0.8311688311688312, 'test_f1': 0.8046266029670606}


In [45]:
for key,value in specific_results.items():
    print(key,value)

test_precision 0.7797270955165692
test_recall 0.8311688311688312
test_f1 0.8046266029670606
